# 作業：調整 ELMo 模型的訓練參數
- ELMo: [Deep contextualized word representations](https://arxiv.org/abs/1802.05365)
- 參考莫煩Python - [ELMo 一詞多義](https://mofanpy.com/tutorials/machine-learning/nlp/elmo/)，並將 Tensorflow 版本改用 PyTorch 實作
***
## [作業目標]
- 調整 ELMo 模型的參數, 觀察 loss 與數據比較

## [參數說明]
- UNITS : LSTM 特徵維度
- N_LAYERS : LSTM 層數
- BATCH_SIZE : 訓練批次大小
- LEARNING_RATE : 學習速率，影響收斂的快慢，須配合 BATCH_SIZE 調整

In [1]:
import os
import time
import torch
from torch import nn

import utils  # this refers to utils.py in https://github.com/MorvanZhou/NLP-Tutorials

In [2]:
class ELMo(nn.Module):
    def __init__(self, vocab_dim, embed_dim, hidden_dim, num_layers, pad_idx=0):
        super(ELMo, self).__init__()
        self.pad_idx = pad_idx

        self.embed = nn.Embedding(vocab_dim, embed_dim, padding_idx=pad_idx)
        nn.init.normal_(self.embed.weight, 0, 0.001)

        # forward lstm
        self.fs = nn.ModuleList([
            nn.LSTM(embed_dim, hidden_dim, batch_first=True) for _ in range(num_layers)
        ])
        self.f_logits = nn.Linear(hidden_dim, vocab_dim)

        # backward lstm
        self.bs = nn.ModuleList([
            nn.LSTM(embed_dim, hidden_dim, batch_first=True) for _ in range(num_layers)
        ])
        self.b_logits = nn.Linear(hidden_dim, vocab_dim)

        self.criterion = nn.CrossEntropyLoss()

    def forward(self, inputs):
        mask = (inputs != self.pad_idx).unsqueeze(-1)
        embedded = self.embed(inputs)  # (batch_size, time_step, embed_dim)

        fxs, bxs = [embedded[:, :-1]], [embedded[:, 1:]]
        for fl, bl in zip(self.fs, self.bs):
            fx, _ = fl(fxs[-1])  # (batch_size, time_step - 1, embed_dim)
            fx = fx * mask[:, :-1]
            fxs.append(fx)

            bx, _ = bl(torch.flip(bxs[-1], [1]))  # (batch_size, time_step - 1, embed_dim)
            bx = torch.flip(bx, [1]) * mask[:, 1:]
            bxs.append(bx)

        return fxs, bxs

    def cal_loss(self, inputs):
        batch_size = inputs.size(0)
        fxs, bxs = self.forward(inputs)
        fo, bo = self.f_logits(fxs[-1]), self.b_logits(bxs[-1])
        loss = (
            self.criterion(fo.view(-1, fo.size(-1)), inputs[:, 1:].contiguous().view(-1)) +
            self.criterion(bo.view(-1, bo.size(-1)), inputs[:, :-1].contiguous().view(-1))
        ) / 2

        return loss, (fo, bo)

    def get_embed(self, inputs):
        fxs, bxs = self.forward(inputs)
        xs = [
            torch.cat((f[:, :-1], b[:, 1:]), axis=-1).cpu().detach().numpy()
            for f, b in zip(fxs, bxs)
        ]

        for x in xs:
            print("layers shape=", x.shape)

        return xs

In [3]:
def train(model, data, step, optimizer, device):
    start_time = time.time()
    for t in range(step):
        inputs = torch.LongTensor(data.sample(BATCH_SIZE)).to(device)
        loss, (fo, bo) = model.cal_loss(inputs)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if t % 100 == 0:
            fp = fo[0].argmax(axis=1)
            bp = bo[0].argmax(axis=1)
            end_time = time.time()
            print(
                f"step: {t} | time: {end_time - start_time:.2f} | loss: {loss.item():.3f}\n",
                f"| tgt: {' '.join([data.i2v[i] for i in inputs[0].tolist() if i != data.pad_id])}\n",
                f"| f_prd: {' '.join([data.i2v[i] for i in fp.tolist() if i != data.pad_id])}\n",
                f"| b_prd: {' '.join([data.i2v[i] for i in bp.tolist() if i != data.pad_id])}\n\n"
            )
            start_time = end_time
    os.makedirs('models', exist_ok=True)
    torch.save(model, os.path.join('models', 'elmo.pt'))

In [4]:
def export_w2v(model, data, device):
    model = torch.load(os.path.join('models', 'elmo.pt'), map_location=device)
    emb = model.get_embed(torch.LongTensor(data.sample(4)).to(device))
    print(emb)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 16
UNITS = 256
N_LAYERS = 2
LEARNING_RATE = 2e-3

data = utils.MRPCSingle('MRPC', rows=2000)
print('num word:', data.num_word)

downloading from https://mofanpy.com/static/files/MRPC/msr_paraphrase_train.txt
completed
downloading from https://mofanpy.com/static/files/MRPC/msr_paraphrase_test.txt
completed
num word: 12880


In [6]:
model = ELMo(
    data.num_word, 
    embed_dim=UNITS, 
    hidden_dim=UNITS, 
    num_layers=N_LAYERS, 
    pad_idx=data.pad_id
).to(device)
model

ELMo(
  (embed): Embedding(12880, 256, padding_idx=0)
  (fs): ModuleList(
    (0): LSTM(256, 256, batch_first=True)
    (1): LSTM(256, 256, batch_first=True)
  )
  (f_logits): Linear(in_features=256, out_features=12880, bias=True)
  (bs): ModuleList(
    (0): LSTM(256, 256, batch_first=True)
    (1): LSTM(256, 256, batch_first=True)
  )
  (b_logits): Linear(in_features=256, out_features=12880, bias=True)
  (criterion): CrossEntropyLoss()
)

In [7]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
train(model, data, 10000, optimizer, device)

step: 0 | time: 0.08 | loss: 9.460
 | tgt: <GO> the vast majority of trades will be priced at <NUM> cents per contract or less depending on participation in incentive schemes . <quote> <SEP>
 | f_prd: people people people people people people people people people people people people people people people people people people people people people people people people people modem modem modem modem modem modem modem modem modem modem modem modem
 | b_prd: represents represents represents represents represents represents represents represents represents represents represents represents represents represents represents represents represents represents represents represents represents represents represents represents cell cell cell cell cell cell cell cell cell cell cell cell cell


step: 100 | time: 6.18 | loss: 7.733
 | tgt: <GO> tony blair has taken a hardline stance arguing nothing should be done to lessen the pressure on mugabe at the gathering in the capital abuja . <SEP>
 | f_prd: th

In [8]:
export_w2v(model, data, device)

layers shape= (4, 36, 512)
layers shape= (4, 36, 512)
layers shape= (4, 36, 512)
[array([[[-3.35995033e-02, -2.55457424e-02, -1.40646234e-01, ...,
          2.48585165e-01,  2.34381646e-01,  5.54705486e-02],
        [ 2.42611319e-01, -9.19304267e-02, -4.45420891e-02, ...,
          1.94007099e-01, -5.48929572e-02,  5.99087738e-02],
        [-5.77322915e-02, -4.22941819e-02,  1.81421209e-02, ...,
          3.30202043e-01, -2.90853769e-01, -1.21969745e-01],
        ...,
        [ 2.17317574e-04,  2.35091639e-03, -6.45548629e-04, ...,
         -4.85155091e-04, -1.12397270e-03, -4.25729144e-04],
        [ 2.17317574e-04,  2.35091639e-03, -6.45548629e-04, ...,
         -4.85155091e-04, -1.12397270e-03, -4.25729144e-04],
        [ 2.17317574e-04,  2.35091639e-03, -6.45548629e-04, ...,
         -4.85155091e-04, -1.12397270e-03, -4.25729144e-04]],

       [[-3.35995033e-02, -2.55457424e-02, -1.40646234e-01, ...,
         -3.11924726e-01, -2.47240901e-01, -1.03219226e-01],
        [-1.47046104e